#### Load in dataset

In [31]:
import pandas as pd
from tabulate import tabulate
import numpy as np
import os
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

#--- Custom Functions as defined by the user ---
import sys
sys.path.append(os.path.abspath("../tools"))
import functions

In [32]:
# Load in dataset
# Update the file path below to the correct location of your CSV file
data = pd.read_csv("../../../../data/finaldatasets/testdata/finalfr/Treetest2.csv", header=0, sep=",")

/tmp/ipykernel_201689/1490542236.py:3: DtypeWarning: Columns (14,15,16,17,20,22,23,24,26,28,29,30,31,32,39,40,42,43,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../../../../data/finaldatasets/testdata/finalfr/Treetest2.csv", header=0, sep=",")


In [33]:
functions.summarize_dataframe(data)

| variable                                              | dtype   |   count |   pct_complete |   n_unique |
|-------------------------------------------------------|---------|---------|----------------|------------|
| id                                                    | int64   |   22709 |          100   |      22315 |
| prc8                                                  | float64 |   22709 |          100   |        270 |
| EvapS1                                                | float64 |   22709 |          100   |        544 |
| Evap9                                                 | float64 |   22709 |          100   |        544 |
| Evap8                                                 | float64 |   22709 |          100   |        544 |
| Evap7                                                 | float64 |   22709 |          100   |        544 |
| Evap6                                                 | float64 |   22709 |          100   |        544 |
| Evap5                     

### Evi

In [34]:
EVIdata = pd.read_csv("../../../../data/finaldatasets/covariates/Earthdata/covs/sebwheatdata-MOD13A3-061-results.csv", header=0, sep=",")

print("Data loaded successfully.")

functions.summarize_dataframe(EVIdata)

Data loaded successfully.
| variable                                                                    | dtype   |   count |   pct_complete |   n_unique |
|-----------------------------------------------------------------------------|---------|---------|----------------|------------|
| ID                                                                          | int64   |   53816 |            100 |        248 |
| Latitude                                                                    | float64 |   53816 |            100 |        237 |
| MOD13A3_061__1_km_monthly_VI_Quality_Possible_shadow                        | object  |   53816 |            100 |          2 |
| MOD13A3_061__1_km_monthly_VI_Quality_Possible_snow/ice_Description          | object  |   53816 |            100 |          2 |
| MOD13A3_061__1_km_monthly_VI_Quality_Possible_snow/ice                      | object  |   53816 |            100 |          2 |
| MOD13A3_061__1_km_monthly_VI_Quality_Land/Water_Mask_Descripti

In [35]:
EVIdata["MOD13A3_061__1_km_monthly_VI_Quality_MODLAND_Description"].value_counts()


MOD13A3_061__1_km_monthly_VI_Quality_MODLAND_Description
VI produced, good quality                              39902
VI produced, but check other QA                        12483
Pixel produced, but most probably cloudy                1397
Pixel not produced due to other reasons than clouds       34
Name: count, dtype: int64

In [36]:
EVIdata = EVIdata[
	~EVIdata["MOD13A3_061__1_km_monthly_VI_Quality_MODLAND_Description"].isin([
		"Pixel produced, but most probably cloudy",
		"Pixel not produced due to other reasons than clouds"
	])
]

In [37]:
EVI = EVIdata[["Date", "Longitude", "Latitude", "MOD13A3_061__1_km_monthly_EVI"]]

In [8]:
functions.summarize_dataframe(EVI)

| variable                      | dtype   |   count |   pct_complete |   n_unique |
|-------------------------------|---------|---------|----------------|------------|
| Date                          | object  |   52385 |            100 |        217 |
| Longitude                     | float64 |   52385 |            100 |        237 |
| Latitude                      | float64 |   52385 |            100 |        237 |
| MOD13A3_061__1_km_monthly_EVI | float64 |   52385 |            100 |       6714 |


In [117]:
EVI.to_csv("EVI.csv", index=False)

In [38]:
def extract_weather_variable(df, evi_df, var_prefix="evi_", window_length=9):
    """
    Extract 9-month EVI windows using merge operations.
    
    Parameters:
    df: Main dataset with 'id', 'sowing_year', 'start_date', coordinates
    evi_df: EVI DataFrame with 'Date', 'Longitude', 'Latitude', 'MOD13A3_061__1_km_monthly_EVI'
    var_prefix: Prefix for output columns
    window_length: Number of months (default 9)
    
    Returns: DataFrame with id, sowing_year, and monthly EVI values
    """
    
    # Prepare EVI data
    evi = evi_df.copy()
    evi['Date'] = pd.to_datetime(evi['Date'])
    evi['year'] = evi['Date'].dt.year
    evi['month'] = evi['Date'].dt.month
    evi['Longitude'] = evi['Longitude'].round(4)
    evi['Latitude'] = evi['Latitude'].round(4)
    
    # Prepare main data
    main = df.copy()
    
    # Create all possible date combinations for each observation
    expanded_rows = []
    for _, row in main.iterrows():
        start_date = datetime(int(row['sowing_year']), int(row['start_date']), 1)
        
        for i in range(window_length):
            window_date = start_date + relativedelta(months=i)
            expanded_rows.append({
                'sowing_year': row['sowing_year'],
                'Conversion.for.longitude': row['Conversion.for.longitude'],
                'Conversion.for.latitude': row['Conversion.for.latitude'],
                'start_date': row['start_date'],
                'window_month': i + 1,
                'target_year': window_date.year,
                'target_month': window_date.month
            })
    
    expanded_df = pd.DataFrame(expanded_rows)
    
    # Merge with EVI data
    merged = expanded_df.merge(
        evi,
        left_on=['Conversion.for.longitude', 'Conversion.for.latitude', 'target_year', 'target_month'],
        right_on=['Longitude', 'Latitude', 'year', 'month'],
        how='left'
    )
    
    # Pivot to get monthly columns
    result = merged.pivot_table(
        index=['sowing_year', 'Conversion.for.longitude', 'Conversion.for.latitude', 'start_date'],
        columns='window_month',
        values='MOD13A3_061__1_km_monthly_EVI',
        aggfunc='first'
    ).reset_index()
    
    # Rename columns
    result.columns = ['sowing_year', 'Conversion.for.longitude', 'Conversion.for.latitude', 'start_date'] + [f"{var_prefix}{i}" for i in range(1, window_length + 1)]
    
    return result

In [39]:
result = extract_weather_variable(data, EVI, var_prefix='evi_', window_length=9)

In [131]:
result.to_csv("EVI.csv", index=False)

In [40]:
data1 = data.merge(result, on=['sowing_year', 'Conversion.for.longitude', 'Conversion.for.latitude', 'start_date'], how='left')

#### FPAR and LAI

In [41]:
LAIANDFPARdata = pd.read_csv("../../../../data/finaldatasets/covariates/Earthdata/covs/sebwheatdata-MCD15A2H-061-results.csv", header=0, sep=",")


In [17]:
functions.summarize_dataframe(data1)

| variable                                              | dtype   |   count |   pct_complete |   n_unique |
|-------------------------------------------------------|---------|---------|----------------|------------|
| Unnamed: 0                                            | int64   |   22709 |          100   |      22709 |
| Trans2                                                | float64 |   22709 |          100   |        537 |
| Evap6                                                 | float64 |   22709 |          100   |        544 |
| Evap5                                                 | float64 |   22709 |          100   |        544 |
| Evap4                                                 | float64 |   22709 |          100   |        544 |
| Evap3                                                 | float64 |   22709 |          100   |        544 |
| Evap2                                                 | float64 |   22709 |          100   |        544 |
| Trans1                    

In [18]:
LAIANDFPARdata["MCD15A2H_061_FparLai_QC_MODLAND_Description"].value_counts()

MCD15A2H_061_FparLai_QC_MODLAND_Description
Good quality (main algorithm with or without saturation)    154355
Other Quality (back-up algorithm or fill values)             51403
Name: count, dtype: int64

In [42]:
LAIANDFPARdata = LAIANDFPARdata[
	~LAIANDFPARdata["MCD15A2H_061_FparLai_QC_MODLAND_Description"].isin([
		"Other Quality (back-up algorithm or fill values)"
	])
]

In [43]:
# Select relevant columns from LAIANDFPARdata
LAIANDFPAR = LAIANDFPARdata[["Date", "Longitude", "Latitude", "MCD15A2H_061_Lai_500m", "MCD15A2H_061_Fpar_500m"]]

In [44]:
functions.summarize_dataframe(LAIANDFPAR)

| variable               | dtype   |   count |   pct_complete |   n_unique |
|------------------------|---------|---------|----------------|------------|
| Date                   | object  |  154355 |            100 |        830 |
| Longitude              | float64 |  154355 |            100 |        194 |
| Latitude               | float64 |  154355 |            100 |        195 |
| MCD15A2H_061_Lai_500m  | float64 |  154355 |            100 |         69 |
| MCD15A2H_061_Fpar_500m | float64 |  154355 |            100 |         94 |


In [139]:
LAIANDFPAR.to_csv("LAIANDFPAR.csv", index=False)

In [45]:
def extract_lai_fpar_variable_fast_exact(df, lai_fpar_df, var_prefix="lai_", window_length=34, variable_col='MCD15A2H_061_Lai_500m'):
    """
    Extract LAI/FPAR windows using nearest-day (±4 days) coordinate-date matches.

    Parameters:
    df: Main dataset with 'sowing_year', 'start_date', and coordinate columns
    lai_fpar_df: LAI/FPAR DataFrame with 'Date', 'Longitude', 'Latitude', and variable_col
    var_prefix: Prefix for output columns (e.g., "lai_", "fpar_")
    window_length: Number of 8-day periods (default 34 ≈ 9 months)
    variable_col: Column name for the LAI/FPAR variable

    Returns: DataFrame with sowing_year, coordinates, and extracted values
    """

    print(f"Processing {len(df)} observations with {len(lai_fpar_df)} satellite records...")

    # 1. PREPARE DATA
    lai_data = lai_fpar_df.copy()
    lai_data['Date'] = pd.to_datetime(lai_data['Date'])

    main = df.copy()
    main['start_datetime'] = pd.to_datetime(
        main['sowing_year'].astype(str) + '-' +
        main['start_date'].astype(str).str.zfill(2) + '-01'
    )

    # 2. CREATE FAST LOOKUP TABLE
    lai_lookup = lai_data.set_index(['Longitude', 'Latitude', 'Date'])[variable_col].to_dict()

    # 3. PROCESS EACH OBSERVATION
    print("Extracting values...")
    results = []

    for _, row in main.iterrows():
        lon = row['Conversion.for.longitude']
        lat = row['Conversion.for.latitude']
        start_date = row['start_datetime']

        result_row = {
            'sowing_year': row['sowing_year'],
            'Conversion.for.longitude': lon,
            'Conversion.for.latitude': lat,
            'start_date': row['start_date']
        }

        # Generate 34 target dates, 8 days apart
        for i in range(1, window_length + 1):
            target_date = start_date + timedelta(days=(i - 1) * 8)

            # Check for nearest available value within ±4 days
            nearest_val = np.nan
            for delta in range(-4, 5):  # -4 to +4 inclusive
                shifted_date = target_date + timedelta(days=delta)
                key = (lon, lat, shifted_date)
                if key in lai_lookup:
                    nearest_val = lai_lookup[key]
                    break

            result_row[f'{var_prefix}{i}'] = nearest_val

        results.append(result_row)

    # 4. RETURN RESULTS
    print("Creating final DataFrame...")
    result_df = pd.DataFrame(results)

    id_cols = ['sowing_year', 'Conversion.for.longitude', 'Conversion.for.latitude', 'start_date']
    data_cols = [f'{var_prefix}{i}' for i in range(1, window_length + 1)]
    result_df = result_df[id_cols + data_cols]

    print(f"✓ Complete! Processed {len(result_df)} observations with {window_length} time steps each.")
    return result_df

In [46]:
#Run the extraction functions

LAI = extract_lai_fpar_variable_fast_exact(data, LAIANDFPAR, var_prefix="lai_", window_length=34, variable_col='MCD15A2H_061_Lai_500m')

FPAR = extract_lai_fpar_variable_fast_exact(data, LAIANDFPAR, var_prefix="fpar_", window_length=34, variable_col='MCD15A2H_061_Fpar_500m')


FPAR.to_csv("FPAR.csv", index=False)

LAI.to_csv("LAI.csv", index=False)

Processing 22709 observations with 154355 satellite records...
Extracting values...
Creating final DataFrame...
✓ Complete! Processed 22709 observations with 34 time steps each.
Processing 22709 observations with 154355 satellite records...
Extracting values...
Creating final DataFrame...
✓ Complete! Processed 22709 observations with 34 time steps each.


CREATING MONTHLY WINDOWS FOR EASY JOINING

In [47]:
def collapse_time_windows(df, var_prefix='lai_', n_bins=8):
    """
    Collapse 34 LAI/FPAR windows into ~monthly averages.

    Parameters:
    - df: DataFrame with columns like 'lai_1' to 'lai_34' or 'fpar_1' to 'fpar_34'
    - var_prefix: 'lai_' or 'fpar_'
    - n_bins: number of collapsed bins (default = 8)

    Returns:
    - DataFrame with collapsed columns named like 'lai_month_1' to 'lai_month_8'
    """
    window_cols = [f"{var_prefix}{i}" for i in range(1, 35)]
    collapsed = pd.DataFrame(index=df.index)

    group_size = int(np.ceil(len(window_cols) / n_bins))

    for i in range(n_bins):
        group = window_cols[i * group_size: (i + 1) * group_size]
        collapsed[f"{var_prefix}month_{i+1}"] = df[group].mean(axis=1)

    return pd.concat([df.drop(columns=window_cols), collapsed], axis=1)

In [48]:
LAI_monthly = collapse_time_windows(LAI, var_prefix='lai_', n_bins=8)
FPAR_monthly = collapse_time_windows(FPAR, var_prefix='fpar_', n_bins=8)

In [49]:
LAI_monthly = LAI_monthly.drop_duplicates()
FPAR_monthly = FPAR_monthly.drop_duplicates()

In [152]:
FPAR_monthly.to_csv("FPAR_monthly.csv", index=False)

LAI_monthly.to_csv("LAI_monthly.csv", index=False)

In [148]:
functions.summarize_dataframe(LAI_monthly)

| variable                 | dtype   |   count |   pct_complete |   n_unique |
|--------------------------|---------|---------|----------------|------------|
| sowing_year              | int64   |   69430 |          100   |         15 |
| Conversion.for.longitude | float64 |   69430 |          100   |        237 |
| Conversion.for.latitude  | float64 |   69430 |          100   |        237 |
| start_date               | int64   |   69430 |          100   |          7 |
| lai_month_6              | float64 |   67871 |           97.8 |        181 |
| lai_month_7              | float64 |   67871 |           97.8 |        185 |
| lai_month_1              | float64 |   67866 |           97.7 |        169 |
| lai_month_5              | float64 |   67758 |           97.6 |        163 |
| lai_month_4              | float64 |   67456 |           97.2 |        161 |
| lai_month_3              | float64 |   67380 |           97   |        140 |
| lai_month_2              | float64 |   65100 |    

In [50]:
data2 = data1.merge(LAI_monthly, on=['sowing_year', 'Conversion.for.longitude', 'Conversion.for.latitude', 'start_date'], how='left')

In [28]:
functions.summarize_dataframe(data2)

| variable                                              | dtype   |   count |   pct_complete |   n_unique |
|-------------------------------------------------------|---------|---------|----------------|------------|
| Unnamed: 0                                            | int64   |   22709 |          100   |      22709 |
| Trans5                                                | float64 |   22709 |          100   |        490 |
| latitude_jittered                                     | float64 |   22709 |          100   |      22709 |
| jitter_radius_km                                      | float64 |   22709 |          100   |         81 |
| group_id                                              | object  |   22709 |          100   |        554 |
| gdp_per_capita                                        | int64   |   22709 |          100   |        486 |
| EvapS2                                                | float64 |   22709 |          100   |        544 |
| EvapS3                    

In [51]:
data3 = data2.merge(FPAR_monthly, on=['sowing_year', 'Conversion.for.longitude', 'Conversion.for.latitude', 'start_date'], how='left')

In [52]:
data3.to_csv("../../../../data/finaldatasets/testdata/finalfr/Treetest2.csv", index=False)